In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.1-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.1-py3-none-any.whl (238 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4246764 sha256=c2d9b2957a72f66482f2f83bb438bcc574450accc23452e6f45df5f193602156
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:

# # for pretrain
# # fastext webseite->multilingual->english bin or any langauge bin
# # directory mrin kahan model hai wo likho
# # it is english bin
# import fasttext
# fasttext.util.download_model('en', if_exists='ignore') # English
# # import fasttext.util
import fasttext
import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore') # English

# # Load the FastText model
# model = fasttext.load_model('cc.en.300.bin')

# # Use the model
# words = model.get_words()
# print(words[:10])  # Print the first 10 words



In [ ]:
fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

In [ ]:
ft = fasttext.load_model('cc.en.300.bin')
ft.get_dimension()

300

In [ ]:
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
100% 25.7M/25.7M [00:02<00:00, 18.9MB/s]
100% 25.7M/25.7M [00:02<00:00, 11.1MB/s]


In [ ]:
!unzip -o imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
print(pd.__version__)
print(np.__version__)
print(tf.__version__)
print(tf.keras.__version__)


2.1.4
1.26.4
2.17.0
3.4.1


In [ ]:
df= pd.read_csv('IMDB Dataset.csv')

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
import re

def remove_html_tags(text):
    # Remove HTML tags
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    # Remove non-alphanumeric characters except spaces
    text = re.sub(r"[^\w\s]", " ", text)

    return text

# Apply the function to the DataFrame
df['review'] = df['review'].apply(remove_html_tags)


In [ ]:
df['review']=df['review'].str.lower()

In [ ]:
import inflect

# Assuming you want to replace numbers with words in the 'email' column
p = inflect.engine()
def replace_numbers_with_words(text):
    words = []
    for word in text.split():
        # Check if the word is a number
        if word.isdigit():
            # Convert the number to its word representation
            words.append(p.number_to_words(word))
        else:
            words.append(word)
    # Join the words back into a single string
    return ' '.join(words)

# Apply the function to the 'email' column
df['review']=df['review'].apply(replace_numbers_with_words)


In [ ]:
def remove_whitespace(text):
    return  " ".join(text.split())
df['review']=df['review'].apply(remove_whitespace)

In [ ]:
df.review.loc[0:3]

0    one of the other reviewers has mentioned that ...
1    a wonderful little production the filming tech...
2    i thought this was a wonderful way to spend ti...
3    basically there s a family where a little boy ...
Name: review, dtype: object

In [ ]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there s a family where a little boy ...,0
4,petter mattei s love in the time of money is a...,1


In [ ]:
df.to_csv("review.txt", columns=["review"], header=None, index=False)


In [ ]:


def get_text_vector(text, model):
    """
    Convert a text to its FastText vector representation.
    """
    return model.get_sentence_vector(text)

def prepare_data(reviews, model):
    """
    Convert a list of reviews to a matrix of FastText vectors.
    """
    vectors = [get_text_vector(review, model) for review in reviews]
    return np.array(vectors)

# Prepare the data
X = prepare_data(df['review'].tolist(), ft)

X = np.expand_dims(X, axis=1)

In [ ]:
X.shape

(50000, 1, 300)

In [ ]:
print(X)

[[[-1.3408220e-02 -2.7169491e-04  1.0576973e-03 ...  7.6085970e-02
   -1.3582788e-02 -6.2010477e-06]]

 [[-9.7657433e-03 -2.9063104e-03 -1.5541650e-03 ...  7.9469293e-02
   -2.7706454e-02  4.5995768e-03]]

 [[-3.9867018e-03 -1.1954033e-02 -4.0995792e-04 ...  8.6071871e-02
   -1.6960336e-02 -6.0381036e-04]]

 ...

 [[-1.4918878e-02 -2.6784183e-03 -4.1361674e-04 ...  8.6924419e-02
   -7.3047931e-04  1.7272560e-02]]

 [[-1.0349989e-02 -2.4240981e-03  6.3805929e-03 ...  8.2534947e-02
   -8.3527705e-03  2.0047198e-03]]

 [[-2.0636279e-02  1.4142325e-02 -5.1614190e-03 ...  9.1500275e-02
   -2.8852059e-02 -2.2971397e-02]]]


In [ ]:
X.shape[1]

1

In [ ]:
model = Sequential([
     LSTM(units=64, input_shape=(1, 300)),  # input_shape=(timesteps, features)
     Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Example training data
y = np.array(df['sentiment'])

# Train the model
model.fit(X, y, epochs=100, batch_size=128)

Epoch 1/100
391/391 [==============================] - 5s 7ms/step - loss: 0.5618 - accuracy: 0.7301
Epoch 2/100
391/391 [==============================] - 2s 5ms/step - loss: 0.3867 - accuracy: 0.8319
Epoch 3/100
391/391 [==============================] - 2s 4ms/step - loss: 0.3605 - accuracy: 0.8456
Epoch 4/100
391/391 [==============================] - 2s 4ms/step - loss: 0.3487 - accuracy: 0.8518
Epoch 5/100
391/391 [==============================] - 2s 4ms/step - loss: 0.3420 - accuracy: 0.8551
Epoch 6/100
391/391 [==============================] - 2s 4ms/step - loss: 0.3378 - accuracy: 0.8576
Epoch 7/100
391/391 [==============================] - 2s 5ms/step - loss: 0.3352 - accuracy: 0.8589
Epoch 8/100
391/391 [==============================] - 3s 7ms/step - loss: 0.3334 - accuracy: 0.8583
Epoch 9/100
391/391 [==============================] - 2s 4ms/step - loss: 0.3311 - accuracy: 0.8602
Epoch 10/100
391/391 [==============================] - 2s 5ms/step - loss: 0.3301 - accura

In [ ]:
def predict_sentiment(text, model, fasttext_model):
    """
    Predict sentiment of a given text using the sentiment analysis model.
    """
    vector = get_text_vector(text, fasttext_model)

    # Add a batch dimension and a timestep dimension to the vector using np.expand_dims
    vector = np.expand_dims(vector, axis=0)  # Add batch dimension
    vector = np.expand_dims(vector, axis=1)  # Add timestep dimension

    print(f"Vector shape for prediction: {vector.shape}")  # Debugging statement

    # Make a prediction using the trained model
    prediction = model.predict(vector)

    sentiment = 'Positive' if prediction[0] > 0.5 else 'Negative'
    return sentiment

# Example usage
user_input = input("enter ur review:")
sentiment = predict_sentiment(user_input, model, ft)
print(f"The sentiment of the review is: {sentiment}")


enter ur review:movie was boring
Vector shape for prediction: (1, 1, 300)
1/1 [==============================] - 0s 19ms/step
The sentiment of the review is: Negative


In [ ]:
model.save("fastestrmoviewreview.keras")

In [ ]:
loaded_model=tf.keras.models.load_model("fastestrmoviewreview.keras")

In [ ]:
def predict_sentiment(text, model, fasttext_model):
    """
    Predict sentiment of a given text using the sentiment analysis model.
    """
    vector = get_text_vector(text, fasttext_model)

    # Add a batch dimension and a timestep dimension to the vector using np.expand_dims
    vector = np.expand_dims(vector, axis=0)  # Add batch dimension
    vector = np.expand_dims(vector, axis=1)  # Add timestep dimension

    print(f"Vector shape for prediction: {vector.shape}")  # Debugging statement

    # Make a prediction using the trained model
    prediction = loaded_model.predict(vector)

    sentiment = 'Positive' if prediction[0] > 0.5 else 'Negative'
    return sentiment

# Example usage
user_input = input("enter ur review:")
sentiment = predict_sentiment(user_input, model, ft)
print(f"The sentiment of the review is: {sentiment}")


enter ur review:boring
Vector shape for prediction: (1, 1, 300)
1/1 [==============================] - 0s 19ms/step
The sentiment of the review is: Negative
